In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [2]:
from datasets import load_dataset
imdb_dataset=load_dataset('imdb')
imdb_dataset
### unsupervised part is the train and test text with out labels

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [3]:
###download tokenizer and model
from transformers import AutoTokenizer,AutoModelForMaskedLM
model_id='distilbert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(model_id)
model=AutoModelForMaskedLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [28]:
### let's see what is mask language model
### note that this model trained on wikipedia
### we want to finetune it on imdb comments
import torch
text = "This is a great [MASK]."
inputs=tokenizer(text,return_tensors='pt')
logits=model(**inputs).logits
mask_token_id=torch.where(inputs['input_ids']==tokenizer.mask_token_id)[1]
mask_token_logits=logits[0,mask_token_id,:]
top_5_tokens=torch.topk(mask_token_logits,5,dim=1).indices[0].tolist()
for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


In [30]:
###lets chose random from our dataset
sample=imdb_dataset['train'].shuffle(seed=42).select(range(3))
for row in sample:
  print(row['text'])

There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...
This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on the moon" It reminds me of Sin

In [31]:
###preprocessing data
def tokenizer_function(example):
  result=tokenizer(example['text'])
  if tokenizer.is_fast:
    result['word_ids']=[result.word_ids(i) for i in range(len(result['input_ids']))]### we need this part for whole word masking
  return result

In [33]:
tokenized_dataset=imdb_dataset.map(tokenizer_function,
                                   batched=True,
                                   remove_columns=['text','label'])
tokenized_dataset

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [34]:
### we saw an warnings here that say the specified maximum sequence length for this model (720 > 512)
### our model can get a max length od 512 token so we need handel this we do it with chunk
### it's good to under stand max length of our model
tokenizer.model_max_length


512

In [35]:
### we chose our chun size lower than 512
###Note that using a small chunk size can be detrimental in real-world scenarios, so you should use a size that corresponds to the use case you will apply your model to.
### and it very depend on our hardware
chunk_size=256

In [47]:
### for do chunk we need concatenate our text
tokenized_sample=tokenized_dataset['train'][:3]
for idx,sample in enumerate(tokenized_sample['input_ids']):
  print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 363'
'>>> Review 1 length: 304'
'>>> Review 2 length: 133'


In [48]:
concatenated_examples ={
    k:sum(tokenized_sample[k],[]) for k in tokenized_sample.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 800'


In [49]:
chunks={
    k:[t[i:i+chunk_size] for i in range(0,total_length,chunk_size)] for k , t in concatenated_examples.items()
}
for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 256'
'>>> Chunk length: 256'
'>>> Chunk length: 256'
'>>> Chunk length: 32'


In [55]:
def group_text(example):
  #First we concatenat our dataset
  concatenated_examples ={ k: sum(example[k],[]) for k in example.keys()}
  #now we want to drop out last chunk becuse this chunk might not be in the size of our chunk_size and we get an error if we pass it to our model
  total_length=len(concatenated_examples[list(example.keys())[0]])### length of input ids
  total_length=(total_length // chunk_size)*chunk_size # for drop out last chunk if it not equal to chunk size
  result={
      k:[t[i:i+chunk_size] for i in range(0,total_length,chunk_size)] for k , t in concatenated_examples.items()
  }
  result['labels']=result['input_ids'].copy()
  return result

In [56]:
lm_dataset=tokenized_dataset.map(group_text,
                                 batched=True)
lm_dataset

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 30639
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 29946
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61465
    })
})

In [75]:
### we should do random masking for fine tune our model
### note that we can do it with DataCollatorForLanguageModeling but it dosen't do whole word masking
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [58]:
from huggingface_hub import notebook_login

notebook_login()

In [76]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(lm_dataset["train"]) // batch_size
model_name = model_id.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb-whole-word-mask",
    overwrite_output_dir=True,

    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
    remove_unused_columns=False

)

In [77]:
model=AutoModelForMaskedLM.from_pretrained(model_id)

In [78]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=whole_word_masking_data_collator,
    tokenizer=tokenizer,
)

<ipython-input-78-d704cec9ace2>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss


In [81]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Alireza0017/distilbert-base-uncased-finetuned-imdb-whole-word-mask/commit/2f02638598e35ef298a496c96b02a71d335b24e3', commit_message='End of training', commit_description='', oid='2f02638598e35ef298a496c96b02a71d335b24e3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Alireza0017/distilbert-base-uncased-finetuned-imdb-whole-word-mask', endpoint='https://huggingface.co', repo_type='model', repo_id='Alireza0017/distilbert-base-uncased-finetuned-imdb-whole-word-mask'), pr_revision=None, pr_num=None)

In [82]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="Alireza0017/distilbert-base-uncased-finetuned-imdb-whole-word-mask"
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Device set to use cuda:0


In [83]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> this is a great film.
>>> this is a great movie.
>>> this is a great idea.
>>> this is a great adventure.
>>> this is a great show.
